In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read in dataset 1
data = pd.read_excel("dataset_1.xls")

In [3]:
# Remove whitespace from column names
data.columns = list(map(lambda x: x.strip().lower(), data.columns))

In [4]:
data.rhaf.isna().sum()

0

# Data Cleaning

From the initial viewing of the data, we noticed that many instances had missing data for some dates (usually tdate10 through tdate12). For now, we have set those values to NaN. For the non-date values, we imputed values as the column's average.

We also noticed whitespace occurred in the columns. We erased them and lowercased the names for simplicity.

In [5]:
# Example of an instance without data for each date, also shows column formatting
len(data.rhaf[data.rhaf.apply(lambda x: x == '   .')])

51

In [6]:
len(data.rhap[data.rhap.apply(lambda x: x == '   .')])

51

In [7]:
data[~data.rhap.apply(lambda x: x == '   .')]

,state,herd,# ptas,ptam,ptaf,ptap,rha # cows,% w ptas,rham,rhaf,...,fat10,pro10,tdate11,milk11,fat11,pro11,tdate12,milk12,fat12,pro12
0,11,1,581,272,17,11,590,98,29810,1093,...,3.8,3.1,2017-10-19 00:00:00,80,4,3.1,2017-09-19 00:00:00,78,3.8,3.1
1,11,2,319,309,23,12,331,97,30020,1134,...,.,.,.,.,.,.,.,.,.,.
2,11,3,10,-448,-18,-19,132,8,22439,892,...,4.2,3.1,2017-08-31 00:00:00,65,3.7,3.1,2017-07-31 00:00:00,64,3.8,3
3,11,4,54,388,16,11,60,90,23466,821,...,3.4,2.9,.,.,.,.,.,.,.,.
4,11,5,76,658,21,19,80,95,27769,1118,...,4.1,3,2017-09-14 00:00:00,74,4.1,3.1,2017-08-16 00:00:00,72,4,3
5,11,6,38,20,-4,1,75,51,16129,589,...,4.1,3,2017-10-10 00:00:00,43,3.8,3,2017-09-12 00:00:00,41,4,2.9
6,11,7,111,-15,0,-1,176,63,22194,805,...,3.3,3.1,.,.,.,.,.,.,.,.
7,11,8,152,362,19,15,210,72,28424,1044,...,3.9,3.2,2017-10-06 00:00:00,76,3.8,3,2017-09-06 00:00:00,75,3.8,3
8,11,9,41,311,2,8,48,86,21075,763,...,3.3,3.2,2017-10-12 00:00:00,54,4,3,2017-09-12 00:00:00,64,3.3,3
9,11,10,37,-21,7,-1,51,72,16093,591,...,4.2,3.3,2017-10-10 00:00:00,43,3.8,3.1,2017-09-08 00:00:00,46,3.5,3.1


In [8]:
# Remove whitespace from column names
data.columns = list(map(lambda x: x.strip().lower(), data.columns))

# Find values of '  .' and set to NaN
for col in data.columns:
    try:
        data[col][data[col] == '  .'] = np.nan
        data[col][data[col] == ' . '] = np.nan
        data[col][data[col].apply(lambda x: x == '    .')] = \
            (data[col][data[col].apply(lambda x: x != '    .')]).mean()
        if "date" not in col:
            data[col] = data[col].astype(float)
    except:
        continue
        
data = data[~data.rhap.apply(lambda x: x == '   .')]
data = data[~data.rhaf.apply(lambda x: x == '   .')]
#data["rhap"][data.rhap.apply(lambda x: x == '   .')] = data.rhap[data.rhap.apply(lambda x: x != '   .')].mean()
#data["rhaf"][data.rhaf.apply(lambda x: x == '   .')] = data.rhaf[data.rhaf.apply(lambda x: x != '   .')].mean()

# Data Analysis

In [9]:
# Dimensions of data
data.shape

(7868, 59)

In [10]:
# Initial summary statistics
data.describe()

,state,herd,# ptas,ptam,ptaf,ptap,rha # cows,% w ptas,rham,milk1,...,pro9,milk10,fat10,pro10,milk11,fat11,pro11,milk12,fat12,pro12
count,7868.000000,7868.000000,7868.000000,7868.000000,7868.000000,7868.000000,7868.000000,7868.000000,7868.000000,7868.000000,...,6377.000000,5862.000000,5862.000000,5862.000000,5084.000000,5084.000000,5084.000000,3792.000000,3792.000000,3792.000000
mean,31.645018,3948.971022,142.494026,151.996187,9.072572,5.805923,214.635613,62.069268,22967.245933,61.246441,...,3.150902,62.032071,3.875384,3.140873,61.739772,3.805566,3.096755,62.259757,3.779562,3.064030
std,11.873456,2279.585578,355.704442,354.728682,12.943216,10.895686,440.014365,28.295911,4037.126645,12.274327,...,0.134823,11.677153,0.320749,0.137300,11.787988,0.313746,0.127782,11.521145,0.301008,0.122117
min,11.000000,1.000000,1.000000,-2940.000000,-108.000000,-99.000000,2.000000,0.000000,462.000000,2.000000,...,2.500000,15.000000,2.200000,2.500000,17.000000,2.400000,2.500000,15.000000,2.400000,2.500000
25%,23.000000,1976.750000,31.000000,-1.000000,3.000000,1.000000,60.000000,45.000000,20598.000000,54.000000,...,3.100000,55.000000,3.700000,3.100000,54.000000,3.600000,3.000000,55.000000,3.600000,3.000000
50%,31.000000,3947.500000,58.000000,213.000000,11.000000,7.000000,94.000000,71.000000,23328.000000,62.000000,...,3.100000,63.000000,3.900000,3.100000,62.000000,3.800000,3.100000,63.000000,3.800000,3.100000
75%,41.000000,5921.250000,117.000000,371.000000,17.000000,12.000000,186.000000,84.000000,25661.000000,70.000000,...,3.200000,70.000000,4.100000,3.200000,70.000000,4.000000,3.200000,70.000000,3.900000,3.100000
max,74.000000,7919.000000,9789.000000,1234.000000,63.000000,44.000000,9008.000000,250.000000,36275.000000,129.000000,...,3.700000,100.000000,6.600000,3.800000,102.000000,6.200000,3.700000,100.000000,5.900000,3.700000


In [11]:
# Amount of unique states the herds are from
len(data.state.unique())

38

In [12]:
# Amount of unique herds, since each row is a unique herd, should be the same amount
len(data.herd.unique())

7868

# Data Visualizations

# Model Creation

### Prepare training and test data

In [22]:
# Create functions for automation between subjects

def prepare_data(data, subject):
    subject_columns = []
    
    # For each column, test if the column pertains to the subject
    for col in data.columns:
        if subject in col:
            subject_columns.append(col)

    # Return the average along the row of all of the subject columns
    return data.loc[:, subject_columns].mean(axis=1, skipna=True)

def split_data(X, y, test_size=0.3):
    
    # Remove rows where the truth label is NA
    X_temp = X[~y.isna()]
    y_temp = y[~y.isna()]
    
    # Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=test_size)
    
    return X_train, X_test, y_train, y_test

In [23]:
# Prepare relevant columns
subjects = ["milk", "fat", "pro"]
feature_cols = ['# ptas', 'ptam', 'ptaf', 'ptap', 'rha # cows',
                '% w ptas', 'rham', 'rhaf', 'rhap']

# Dictionaries for each subject
y = {}
subject_splits = {}
cv_scores = {}
test_rr = {}

# Training feature vectors
X = data.loc[:, feature_cols]

# Split data into train and test for each subject
for subject in subjects:
    y[subject] = prepare_data(data, subject)
    subject_splits[subject] = split_data(X, y[subject])

### Perform cross-validation to test values of alpha

In [24]:
for subject in subjects:
    # Keep track of cv scores for the current subject
    temp_scores = {}
    
    # For each alpha value, perform cross validation w/ Lasso and keep track of scores
    for a in range(0, 25):
        X_train, _, y_train, _ = subject_splits[subject]
        lasso = Lasso(alpha=a)
        temp_scores[a] = cross_val_score(lasso, X_train, y_train, cv=10).mean()
        
    # Store all scores in dictionary
    cv_scores[subject] = temp_scores

cv_scores

{'milk': {0: 0.9851474437872835,
  1: 0.9851570687319603,
  2: 0.9851567233739932,
  3: 0.9851574871715097,
  4: 0.9851576597827245,
  5: 0.9851566702129576,
  6: 0.9851554730269818,
  7: 0.9851540575578335,
  8: 0.985152426798685,
  9: 0.9851505859230543,
  10: 0.985148528212249,
  11: 0.9851462584569924,
  12: 0.9851437740771714,
  13: 0.9851410717970008,
  14: 0.9851381624124944,
  15: 0.9851350372221261,
  16: 0.985131993296218,
  17: 0.9851294567825205,
  18: 0.9851269015603984,
  19: 0.9851242879599071,
  20: 0.985121571797704,
  21: 0.9851186672904927,
  22: 0.9851155472549806,
  23: 0.9851122891691265,
  24: 0.9851088818568613},
 'fat': {0: 0.9525385477109125,
  1: 0.9486605616202418,
  2: 0.9366591333320443,
  3: 0.9168389404003197,
  4: 0.8891870009794098,
  5: 0.853703262769882,
  6: 0.8103874739783651,
  7: 0.7592399810489638,
  8: 0.7002605596341185,
  9: 0.6334488039541942,
  10: 0.5588047696651228,
  11: 0.47632722890644147,
  12: 0.38601574560877233,
  13: 0.28786608847

As you can see, variations in alpha don't cause much difference in R^2 values for milk. For fat and protein, as alpha increase, our training accuracy decreases significantly. Therefore, we will elect to have a less complex model with alpha = 0 (no regularization).

### Evaluate on test data

In [30]:
# For each subject, fit on train and predict on test
for subject in subjects:
    lasso = Lasso(alpha=0)
    X_train, X_test, y_train, y_test = subject_splits[subject]
    lasso.fit(X_train, y_train)
    
    # Store R^2 values in dictionary
    test_rr[subject] = lasso.score(X_test, y_test)

test_rr

{'milk': 0.9939346823840356,
 'fat': 0.9031682864783066,
 'pro': 0.9460625229059595}